<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 4

###  Big Query, SQL, Classification

---

### The Data

There are 5 individual tables that have the information, contained in a Google BigQuery database.  The setup info for BigQuery is located on our DSI wiki.  You will have to query with SQL, the dataset in order to complete this project.

The tables, with cooresonding attributes that exist are:

### businesses
- business_id: unique business identifier
- name: name of the business
- review_count: number of reviews per business
- city: city business resides in
- stars: average rating
- categories: categories the business falls into (can be one or multiple)
- latitude
- longitude
- neighborhoods: neighborhoods business belongs to
- variable: "property" of the business (a tag)
- value: True/False for the property

### reviews
- user_id: unique user identifier
- review_id: unique review identifier
- votes.cool: how many thought the review was "cool"
- business_id: unique business id the review is for
- votes.funny: how many thought the review was funny
- stars: rating given
- date: date of review
- votes.useful: how many thought the review was useful
- ... 100 columns of counts of most common 2 word phrases that appear in reviews in this review

### users
- yelping_since: signup date
- compliments.plain: # of compliments "plain"
- review_count: # of reviews:
- compliments.cute: total # of compliments "cute"
- compliments.writer: # of compliments "writer"
- compliments.note: # of compliments "note" (not sure what this is)
- compliments.hot: # of compliments "hot" (?)
- compliments.cool: # of compliments "cool"
- compliments.profile: # of compliments "profile"
- average_stars: average rating
- compliments.more: # of compliments "more"
- elite: years considered "elite"
- name: user's name
- user_id: unique user id
- votes.cool: # of votes "cool"
- compliments.list: # of compliments "list"
- votes.funny: # of compliments "funny"
- compliments.photos: # of compliments "photos"
- compliments.funny: # of compliments "funny"
- votes.useful: # of votes "useful"

### checkins
- business_id: unique business identifier
- variable: day-time identifier of checkins (0-0 is Sunday 0:00 - 1:00am,  for example)
- value: # of checkins at that time

### tips
- user_id: unique user identifier
- business_id: unique business identifier
- likes: likes that the tip has
- date: date of tip
- ... 100 columns of counts of most common 2 word phrases that appear in tips in this tip


The reviews and tips datasets in particular have parsed "NLP" columns with counts of 2-word phrases in that review or tip (a "tip", it seems, is some kind of smaller review).

The user dataset has a lot of columns of counts of different compliments and votes. We're not sure whether the compliments or votes are by the user or for the user.

Full details about this dataset area located here:
https://bigquery.cloud.google.com/dataset/bigquery-dsi-dave:yelp_arizona

---


If you look at the website, or the full data, you'll see I have removed pieces of the data and cut it down quite a bit. This is to simplify it for this project. Specifically, business are limited to be in these cities:

- Phoenix
- Surprise
- Las Vegas
- Waterloo

Apparently there is a city called "Surprise" in Arizona. 

Businesses are also restricted to at least be in one of the following categories, because we thought the mix of them was funny:

- Airports
- Breakfast & Brunch
- Bubble Tea
- Burgers
- Bars
- Bakeries
- Breweries
- Cafes
- Candy Stores
- Comedy Clubs
- Courthouses
- Dance Clubs
- Fast Food
- Museums
- Tattoo
- Vape Shops
- Yoga
    
---

### Project requirements

**You will be performing 4 different sections of analysis, like in the last project.**

Remember that classification targets are categorical and regression targets are continuous variables.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load your dataset(s) / setup / configure GBQ connection

---

Information about this dataset is located here:


**If you haven't done so, setup a project with the Google developer portal, following the directions here: [Getting Started with BigQuery](https://github.com/ga-students/DSI-SF-4/wiki/Getting-Started-with-BigQuery)**

In [934]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

%matplotlib inline

In [935]:
project_id = "bigquery-dsi-davis"

sql_biz = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.businesses] 
LIMIT 12000
"""

sql_checks = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.checkins] 
LIMIT 12000
"""

sql_revs = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.reviews] 
LIMIT 12000
"""

sql_tips = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.tips] 
LIMIT 12000
"""

sql_users = """
SELECT * FROM [bigquery-dsi-dave:yelp_arizona.users] 
LIMIT 12000
"""

df_biz = pd.read_gbq(sql_biz, project_id=project_id)
df_checks = pd.read_gbq(sql_checks, project_id=project_id)
df_revs = pd.read_gbq(sql_revs, project_id=project_id)
df_tips = pd.read_gbq(sql_tips, project_id=project_id)
df_users = pd.read_gbq(sql_users, project_id=project_id)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 12000 rows.

Total time taken 3.64 s.
Finished at 2016-12-20 09:49:11.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 12000 rows.

Total time taken 2.34 s.
Finished at 2016-12-20 09:49:14.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 1; 27.0% done. Elapsed 7.49 s.
  Got page: 2; 54.0% done. Elapsed 13.18 s.
  Got page: 3; 81.0% done. Elapsed 18.65 s.
  Got page: 4; 100.0% done. Elapsed 22.53 s.
Got 12000 rows.

Total time taken 26.21 s.
Finished at 2016-12-20 09:49:40.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 1; 28.0% done. Elapsed 7.92 s.
  Got page: 2; 56.0% done. Elapsed 12.85 s.
  Got page: 3; 84.0% done. Elapsed 18.32 s.
  Got page: 4; 100.0% done. Elapsed 21.33 s.
Got 12000 rows.

Total time taken 24.68 s.
Finished at 2016-12-20 09:50:05

In [936]:
print df_biz.shape, df_checks.shape, df_revs.shape, df_tips.shape, df_users.shape

(12000, 11) (12000, 3) (12000, 108) (12000, 104) (12000, 21)


In [937]:
df_revs.head()

,user_id,review_id,votes_cool,business_id,votes_funny,stars,date,votes_useful,minutes_10,minutes_15,...,service_great,staff_friendly,super_friendly,sweet_potato,tasted_like,time_vegas,try_place,ve_seen,ve_tried,wait_staff
0,fPHLPrymsyb6WSFFKoMrTQ,3vX3VkmLV1SGSFEwzJbCqg,1,t1sCvl0nSbWeRj0ZgsolRA,2,4,2010-12-07,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tbItoBqYw5ANjrRVsyWCdA,6rV3DZKmIRcTVpKJbCZCBg,0,4bEjOyTaDG24SY5TxsaUNQ,0,4,2010-03-29,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6eEfoYQRwlrLcsBVWxeCfg,rGxZ4Uq8ihaCHT28LziNMw,0,4bEjOyTaDG24SY5TxsaUNQ,1,4,2013-04-14,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KQGdMxKtRtIVs1WGRkA1VQ,Puaz1nM7Jlq8GrY4DBt3-g,1,4bEjOyTaDG24SY5TxsaUNQ,1,4,2013-09-07,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,H1bnxc4KqxQ0RNtn8kyS1A,bg9TTcGaV992e1GUG32law,0,4bEjOyTaDG24SY5TxsaUNQ,0,4,2013-09-30,1,0,0,...,0,0,0,0,0,0,0,0,0,0


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Constructing a "profile" for Las Vegas

---

Yelp is interested in building out what they are calling "profiles" for cities. They want you to start with just Las Vegas to see what a prototype of this would look like. Essentially, they want to know what makes Las Vegas distinct from the other four.

Use the data you have to predict Las Vegas from the other variables you have. You should not be predicting the city from any kind of location data or other data perfectly associated with that city (or another city).

You may use any classification algorithm you deem appropriate, or even multiple models. You should:

1. Build at least one model predicting Las Vegas vs. the other cities.
- Validate your model(s).
- Interpret and visualize, in some way, the results.
- Write up a "profile" for Las Vegas. This should be a writeup converting your findings from the model(s) into a human-readable description of the city.

*Research location data to find the city targets.*

In [938]:
###From my results it looks like there are some determining coeficients that can predict the profile of Vegas
###(minus the vegas specific ones). For example, if I was to start a restaurant in Vegas, I'd want a few things
###that appear to be popular like "Smoking" and "not great for kids"

In [939]:
#df_biz.head()
#df_biz['neighborhoods'].unique()
####remove location and city specific data
df_biz.drop(['latitude','longitude','neighborhoods'], axis=1, inplace=True)

In [940]:
df_biz.head()
df_biz.shape

(12000, 8)

In [941]:
df_biz['name'].unique()
df_biz['city'].unique()
df_biz['variable'].unique()
sorted(df_biz['variable'].unique())
df_biz['variable'].value_counts()

open                                           330
attributes.Accepts Credit Cards                316
attributes.Price Range                         306
attributes.Good For Groups                     272
attributes.Parking.garage                      267
attributes.Parking.street                      266
attributes.Parking.valet                       266
attributes.Parking.lot                         266
attributes.Parking.validated                   266
attributes.Outdoor Seating                     260
attributes.Good for Kids                       244
attributes.Alcohol                             242
attributes.Has TV                              241
attributes.Delivery                            232
attributes.Take-out                            225
attributes.Takes Reservations                  224
attributes.Noise Level                         222
attributes.Attire                              222
attributes.Good For.brunch                     219
attributes.Good For.dinner     

In [942]:
df_biz.head()

,business_id,name,review_count,city,stars,categories,variable,value
0,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",attributes.Takes Reservations,False
1,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",attributes.Good For.dessert,False
2,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",attributes.Take-out,True
3,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",attributes.Has TV,False
4,--jFTZmywe7StuZ2hEjxyA,Subway,7,Las Vegas,3.5,"['Fast Food', 'Sandwiches', 'Restaurants']",attributes.Good For.breakfast,True


In [943]:
index_biz = ['business_id','name','review_count','city','stars']

df_biz_pivot = pd.pivot_table(df_biz, values=['value'], columns=['variable'], index=index_biz)
df_biz_pivot.head()
df_biz_pivot = df_biz_pivot['value'].reset_index()

In [944]:
# def boolean_change(df):
#     for x in df:
#         if x == "True":
#             return 1
#         elif x =="False":
#             return 0
        

# temp_list = ['variable','business_id','name', 'review_count','city','stars']
# for col in df_biz_pivot.columns:
#     if col not in temp_list:
#         df_biz_pivot.col = df_biz_pivot.col.map(boolean_change)

In [945]:
##Tips data - Has extra review or "tips" information that can be combined with the 
##business data to give more details

In [946]:
##Categorical columns are represented in binary (0,1)
df_tips['hours_24'].unique()
df_tips['amazing_food'].unique()

array([0, 1])

In [947]:
##Want to join the business and tips data set on the business id,
##so I can remove the unecessary columns and format the information
##according to 'business_id'
tips_delete = ['user_id', 'likes', 'date']
df_tips_biz = df_tips.drop(tips_delete, axis=1)
df_tips_biz = df_tips_biz.groupby('business_id').agg(np.sum).reset_index()

In [948]:
##Add indicator to the end of each label for clarity once the data is combined
def update_columns(col_names, label):
    new_names = []
    new_names.append(col_names[0])
    for name in col_names[1:]:
        name = label + name
        new_names.append(name)
    return new_names

df_tips_biz.columns = update_columns(df_tips_biz.columns, 'tips_')

In [949]:
##Review Data - repeating the same process as above that I did for the tips data set
##so I can group more relevant information by 'business_id' and then combine it with the
##business data set
revs_delete = ['user_id', 'review_id', 'votes_cool', 'votes_funny', 'stars', 'date', 'votes_useful']
df_revs_biz = df_revs.drop(revs_delete, axis=1)
df_revs_biz = df_revs_biz.groupby('business_id').agg(np.sum).reset_index()
df_revs_biz.columns = update_columns(df_revs_biz.columns, 'revs_')

In [950]:
##3 Tables to merge - "df_revs_biz", "df_tips_biz", "df_biz_pivot"
print df_biz_pivot.shape, df_revs_biz.shape, df_tips_biz.shape
df_biz_comb = pd.merge(df_biz_pivot, df_revs_biz, how='outer', on='business_id')
print df_biz_comb.shape
df_biz_comb = pd.merge(df_biz_comb, df_tips_biz, how='outer', on='business_id')
print df_biz_comb.shape
df_tips_revs_comb = pd.merge(df_revs_biz, df_tips_biz, how='outer', on='business_id')
print df_tips_revs_comb.shape

(330, 83) (2339, 100) (484, 101)
(2484, 182)
(2584, 282)
(2539, 200)


In [951]:
###I have more columns than rows, so I have to many variables that will create too much
###So I will reduce the number of columns in my data set and only look at one data set for
###for right now.

# df_tips_revs_comb = pd.merge(df_revs_biz, df_tips_biz, how='outer', on='business_id')
# df_tips_revs_comb.shape

###Actually found I was merging the tables incorrectly, so changed it to 'outer' rather than 'inner' and now
###I have much more data than previously

In [952]:
df_biz_comb['city'] = df_biz_comb['city'].map(lambda x: 1 if x == 'Las Vegas' else 0)
df_biz_comb = df_biz_comb.drop(['business_id', 'name', 'review_count'], axis=1)

In [953]:
for x in df_biz_comb.columns:
    if 'attributes' in x:
        df_biz_comb[x] = df_biz_comb[x].map(lambda x: 1 if x == True else 0)

In [954]:
df_biz_comb.columns = [x.strip().replace(' ','_').replace('.','_').replace('-','_').replace('/','_') for x in df_biz_comb.columns]

In [955]:
hours_list = []
for x in df_biz_comb.columns:
    if 'hours' in x:
        hours_list.append(x)

In [956]:
remove_list = ['city'] + hours_list
formula = 'city ~ ' + ' + '.join([x for x in df_biz_comb.columns if x not in remove_list]) + ' -1'
print formula

city ~ stars + attributes_Accepts_Credit_Cards + attributes_Accepts_Insurance + attributes_Ages_Allowed + attributes_Alcohol + attributes_Ambience_casual + attributes_Ambience_classy + attributes_Ambience_divey + attributes_Ambience_hipster + attributes_Ambience_intimate + attributes_Ambience_romantic + attributes_Ambience_touristy + attributes_Ambience_trendy + attributes_Ambience_upscale + attributes_Attire + attributes_BYOB + attributes_BYOB_Corkage + attributes_By_Appointment_Only + attributes_Caters + attributes_Coat_Check + attributes_Corkage + attributes_Delivery + attributes_Dietary_Restrictions_dairy_free + attributes_Dietary_Restrictions_gluten_free + attributes_Dietary_Restrictions_halal + attributes_Dietary_Restrictions_kosher + attributes_Dietary_Restrictions_soy_free + attributes_Dietary_Restrictions_vegan + attributes_Dietary_Restrictions_vegetarian + attributes_Dogs_Allowed + attributes_Drive_Thru + attributes_Good_For_Dancing + attributes_Good_For_Groups + attributes_G

In [957]:
import patsy
y,X = patsy.dmatrices(formula, data=df_biz_comb, return_type='dataframe')

In [958]:
from sklearn.preprocessing import StandardScaler
y = np.ravel(y)
#Standardize the data becuase it's not all binary
ss = StandardScaler()
Xn = ss.fit_transform(X)

In [959]:
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

#Ridge works best going through logspace to find alphas

lr_params = {
    'penalty':['l1','l2'],
    'solver':['liblinear'],
    'C':np.logspace(0, 5, 200),

}

vegas_lr_gs = GridSearchCV(LogisticRegression(), lr_params, cv=5, verbose=1)
vegas_lr_gs.fit(Xn, y)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:   16.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1.00000e+00,   1.05956e+00, ...,   9.43788e+04,   1.00000e+05]), 'solver': ['liblinear']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [960]:
print vegas_lr_gs.best_score_
print vegas_lr_gs.best_estimator_
print np.mean(y)
print vegas_lr_gs.best_params_

0.809523809524
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.142857142857
{'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear'}


In [961]:
vegas_model = vegas_lr_gs.best_estimator_

from sklearn.cross_validation import cross_val_score
model_scores = cross_val_score(vegas_model, Xn, y, cv=5)
print model_scores
print np.mean(model_scores)

[ 0.84615385  0.76923077  0.73076923  0.83333333  0.875     ]
0.810897435897


In [962]:
vegas_model.fit(Xn, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [963]:
coefs = pd.DataFrame({'coef':vegas_model.coef_[0], 'abs_coef':np.abs(vegas_model.coef_[0]),
                      'variable':X.columns})
coefs.sort_values('abs_coef', ascending=False, inplace=True)
coefs.head(20)

,abs_coef,coef,variable
60,1.008172,1.008172,attributes_Smoking
241,0.886919,0.886919,tips_nom_nom
54,0.836906,0.836906,attributes_Parking_garage
41,0.504566,-0.504566,attributes_Good_for_Kids
174,0.454962,-0.454962,tips_best_burgers
108,0.447782,0.447782,revs_great_experience
214,0.410797,-0.410797,tips_great_burgers
121,0.410182,0.410182,revs_las_vegas
7,0.397341,0.397341,attributes_Ambience_casual
243,0.393074,0.393074,tips_open_24


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Different categories of ratings

---

Yelp is finally ready to admit that their rating system sucks. No one cares about the ratings, they just use the site to find out what's nearby. The ratings are simply too unreliable for people. 

Yelp hypothesizes that this is, in fact, because different people tend to give their ratings based on different things. They believe that perhaps some people always base their ratings on quality of food, others on service, and perhaps other categories as well. 

1. Do some users tend to talk about service more than others in reviews/tips? Divide up the tips/reviews into more "service-focused" ones and those less concerned with service.
2. Create two new ratings for businesses: ratings from just the service-focused reviews and ratings from the non-service reviews.
3. Construct a regression model for each of the two ratings. They should use the same predictor variables (of your choice). 
4. Validate the performance of the models.
5. Do the models coefficients differ at all? What does this tell you about the hypothesis that there are in fact two different kinds of ratings?

In [ ]:
###Looking at my outputs, it does appear that service is inversily correlated against the non-service
###descriptions. This means that it's important to have good service beccause higher "like" ratings are more
###influenced by higher service ratings

In [964]:
#df_revs, df_tips

In [965]:
def service_check(df_col):
    service_list = []
    for x in df_col:
        if 'service' in x or 'staff' in x or 'people' in x:
            service_list.append(x)
    return service_list

In [966]:
df_tips_prob_two = df_tips
tips_delete = ['date']
df_tips_prob_two = df_tips_prob_two.drop(tips_delete, axis=1)

In [967]:
df_tips_prob_two.shape

(12000, 103)

In [968]:
tips_columns = df_tips_prob_two.columns[3:]
df_tips_prob_two['total_tips'] = df_tips_prob_two[tips_columns].sum(axis=1)
df_tips_prob_two['total_tips'].unique()

array([3, 1, 0, 2, 5, 4, 6])

In [969]:
df_tips_prob_two.shape

(12000, 104)

In [970]:
tips_service_list = service_check(df_tips_prob_two.columns)
df_tips_prob_two['service_counted'] = df_tips_prob_two[tips_service_list].sum(axis=1)
df_tips_prob_two['service_counted'].unique()

array([1, 0, 2, 3])

In [971]:
df_tips_prob_two.shape

(12000, 105)

In [972]:
###No common predictor variable between the two tables so going to use the "tips" table and use the likes as my
###target variable. Assuming I can find the average for each business, the business that have more likes are probably
###realated to helpful hints (i.e. good service or bad service)

###Using likes didn't work becuase not enough data is available to predict on, so I'm going to make
###a new column that just looks at the total number of comments/likes
df_tips_prob_two['service_check'] = df_tips_prob_two['service_counted'].map(lambda x: 1 if x > 0 else 0)

df_service = df_tips_prob_two[df_tips_prob_two['service_check'] == 1]
df_non_service = df_tips_prob_two[df_tips_prob_two['service_check'] == 0]

###find what the average business id likes is with and without service tips
# avg_df_service = df_service.groupby('business_id')['service_counted'].mean().reset_index()
# avg_df_non_service = df_non_service.groupby('business_id')['service_counted'].mean().reset_index()

In [973]:
df_tips_prob_two.shape

(12000, 106)

In [974]:
print df_tips_prob_two.shape, df_service.shape, df_non_service.shape

(12000, 106) (647, 106) (11353, 106)


In [975]:
#Pick predictor's and matrix for both types of table
col_not_used = ['service_check', 'service_counted', 'total_tips', 'user_id', 'business_id']

y_service = df_service['total_tips']
X_service = df_service[[x for x in df_service.columns if x not in col_not_used]]

y_non_service = df_non_service['total_tips']
X_non_service = df_non_service[[x for x in df_non_service.columns if x not in col_not_used]]


ss = StandardScaler()
Xn_service = ss.fit_transform(X_service)
Xn_non_service = ss.fit_transform(X_non_service)

#Look at the elastic net to find the best fit
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_validation import cross_val_score

l1_ratios = np.linspace(0.01, 1.0, 50)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=300, cv=5,
                            verbose=1)
optimal_enet.fit(Xn_service, y_service)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=array([ 0.01   ,  0.0302 ,  0.05041,  0.07061,  0.09082,  0.11102,
        0.13122,  0.15143,  0.17163,  0.19184,  0.21204,  0.23224,
        0.25245,  0.27265,  0.29286,  0.31306,  0.33327,  0.35347,
        0.37367,  0.39388,  0.41408,  0.43429,  0.45449,  0.47469,
        0.4949 ,  0.515...3837,
        0.85857,  0.87878,  0.89898,  0.91918,  0.93939,  0.95959,
        0.9798 ,  1.     ]),
       max_iter=1000, n_alphas=300, n_jobs=1, normalize=False,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=1)

In [976]:
print optimal_enet.alpha_
print optimal_enet.l1_ratio_
### So going to go with lasso

0.000348277472763
1.0


In [977]:
###Service model - Lasso Fit
lcv_service = LassoCV(n_alphas=1000, cv=10, verbose=1)
lcv_service.fit(Xn_service, y_service)
lasso_service = Lasso(alpha=lcv_service.alpha_)
lasso_service.fit(Xn_service, y_service)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Lasso(alpha=0.0003482774727627444, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [978]:
service_scores = cross_val_score(lasso_service, Xn_service, y_service, cv=5)
print service_scores
print np.mean(service_scores)

[ 0.95375608  0.96790529  0.95373471  0.94129621  0.89163651]
0.941665761


In [979]:
###Non-Service model - Lasso Fit
lcv_non_service = LassoCV(n_alphas=1000, cv=10, verbose=1)
lcv_non_service.fit(Xn_non_service, y_non_service)
lasso_non_service = Lasso(alpha=lcv_non_service.alpha_)
lasso_non_service.fit(Xn_non_service, y_non_service)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Lasso(alpha=0.00013548138537890534, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [980]:
non_service_scores = cross_val_score(lasso_non_service, Xn_non_service, y_non_service, cv=5)
print non_service_scores
print np.mean(non_service_scores)

[ 0.99998212  0.99999177  0.99999271  0.99998752  0.99799255]
0.9995893342


In [981]:
service_coefs = pd.DataFrame({'variable':X_service.columns, 
                      'service_coef':lasso_service.coef_, 
                      'abs_service_coef':np.abs(lasso_service.coef_),
                      })
service_coefs.sort_values(by='service_coef', ascending=False).head(10)

,abs_service_coef,service_coef,variable
58,0.433482,0.433482,great_service
54,0.312531,0.312531,great_food
38,0.308022,0.308022,food_service
43,0.298346,0.298346,friendly_staff
47,0.290686,0.290686,good_service
21,0.279338,0.279338,customer_service
37,0.271264,0.271264,food_great
45,0.254351,0.254351,good_food
92,0.251450,0.251450,service_great
90,0.232338,0.232338,service_food


In [982]:
non_service_coefs = pd.DataFrame({'variable':X_service.columns, 
                      'non_service_coef':lasso_non_service.coef_,
                      'abs_non_service_coef':np.abs(lasso_non_service.coef_)
                      })
non_service_coefs.sort_values(by='non_service_coef', ascending=False).head(10)

,abs_non_service_coef,non_service_coef,variable
54,0.110642,0.110642,great_food
73,0.103392,0.103392,love_place
56,0.087575,0.087575,great_place
45,0.085103,0.085103,good_food
37,0.082037,0.082037,food_great
67,0.075910,0.075910,las_vegas
40,0.071179,0.071179,french_toast
72,0.069410,0.069410,love_love
75,0.069282,0.069282,make_sure
9,0.066073,0.066073,best_breakfast


In [983]:
total_coefs = pd.DataFrame({'variable':X_service.columns, 
                      'service_coef':lasso_service.coef_, 
                      'abs_service_coef':np.abs(lasso_service.coef_),
                      'non_service_coef':lasso_non_service.coef_,
                      'abs_non_service_coef':np.abs(lasso_non_service.coef_)
                      })
total_coefs.sort_values(by='abs_service_coef', ascending=False).head(10)

,abs_non_service_coef,abs_service_coef,non_service_coef,service_coef,variable
58,0.000000,0.433482,0.000000,0.433482,great_service
54,0.110642,0.312531,0.110642,0.312531,great_food
38,0.000000,0.308022,0.000000,0.308022,food_service
43,0.000000,0.298346,0.000000,0.298346,friendly_staff
47,0.000000,0.290686,0.000000,0.290686,good_service
21,0.000000,0.279338,0.000000,0.279338,customer_service
37,0.082037,0.271264,0.082037,0.271264,food_great
45,0.085103,0.254351,0.085103,0.254351,good_food
92,0.000000,0.251450,0.000000,0.251450,service_great
90,0.000000,0.232338,0.000000,0.232338,service_food


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Identifying "elite" users

---

Yelp, though having their own formula for determining whether a user is elite or not, is interested in delving deeper into what differentiates an elite user from a normal user at a broader level.

Use a classification model to predict whether a user is elite or not. Note that users can be elite in some years and not in others.

1. What things predict well whether a user is elite or not?
- Validate the model.
- If you were to remove the "counts" metrics for users (reviews, votes, compliments), what distinguishes an elite user, if anything? Validate the model and compare it to the one with the count variables.
- Think of a way to visually represent your results in a compelling way.
- Give a brief write-up of your findings.


In [ ]:
###Unfortunately my classification models took a little too much time and I couldn't define the specific coefficients
###that defined an "elite" user. With more time, I would look the number of reviews a user wrote, look a the number of
###comments they make, or how high their average "like" and "stars" ratings for business would be. My assumption is that
###they could be viewed as elite by the amount of activity they had. I would then plot an ROC curve to see how
###accurate my predictions are against their actual "elite" status (if I had a little more time to finish...)

In [984]:
df_users.head()

,yelping_since,compliments_plain,review_count,compliments_cute,compliments_writer,fans,compliments_note,compliments_hot,compliments_cool,compliments_profile,...,compliments_more,elite,name,user_id,votes_cool,compliments_list,votes_funny,compliments_photos,compliments_funny,votes_useful
0,2004-10,NaN,6,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,[],Brandon,GwnFpsaEaLZfVh844EM0Jg,1,None,0,None,NaN,7
1,2004-10,1.0,5,NaN,1.0,1,NaN,NaN,NaN,NaN,...,NaN,[],Teel,5KwFISX_GfnFUQ0VvaIE1A,1,None,2,None,NaN,16
2,2004-10,1.0,94,NaN,1.0,1,1.0,NaN,NaN,NaN,...,1.0,[],Larry,82d-mm3hB2UelHwrptvcMw,8,None,4,None,NaN,17
3,2004-10,2.0,66,2.0,2.0,4,1.0,1.0,1.0,NaN,...,1.0,[2005],Katherine,SIBCL7HBkrP4llolm4SC2A,13,None,11,None,NaN,34
4,2004-10,89.0,442,23.0,24.0,100,83.0,101.0,145.0,9.0,...,19.0,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",Michael,4U9kSBLuBDU391x6bxU-YA,732,4.0,908,24.0,120.0,1483


In [985]:
#df_users.head()
df_users_elite = df_users
df_users_elite.head()

,yelping_since,compliments_plain,review_count,compliments_cute,compliments_writer,fans,compliments_note,compliments_hot,compliments_cool,compliments_profile,...,compliments_more,elite,name,user_id,votes_cool,compliments_list,votes_funny,compliments_photos,compliments_funny,votes_useful
0,2004-10,NaN,6,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,[],Brandon,GwnFpsaEaLZfVh844EM0Jg,1,None,0,None,NaN,7
1,2004-10,1.0,5,NaN,1.0,1,NaN,NaN,NaN,NaN,...,NaN,[],Teel,5KwFISX_GfnFUQ0VvaIE1A,1,None,2,None,NaN,16
2,2004-10,1.0,94,NaN,1.0,1,1.0,NaN,NaN,NaN,...,1.0,[],Larry,82d-mm3hB2UelHwrptvcMw,8,None,4,None,NaN,17
3,2004-10,2.0,66,2.0,2.0,4,1.0,1.0,1.0,NaN,...,1.0,[2005],Katherine,SIBCL7HBkrP4llolm4SC2A,13,None,11,None,NaN,34
4,2004-10,89.0,442,23.0,24.0,100,83.0,101.0,145.0,9.0,...,19.0,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",Michael,4U9kSBLuBDU391x6bxU-YA,732,4.0,908,24.0,120.0,1483


In [986]:
df_users_elite['elite'].values

array([u'[]', u'[]', u'[]', ..., u'[2010, 2011, 2012, 2013, 2014, 2015]',
       u'[2009, 2011, 2012, 2013, 2014, 2015]',
       u'[2009, 2010, 2011, 2012, 2013, 2014]'], dtype=object)

In [987]:
#df_users_elite.drop(['elite_status','temp_elite'], axis=1, inplace=True)
#df_users_elite.drop('temp_elite_len', axis=1, inplace=True)
#df_users_elite.drop('temp_elite', axis=1, inplace=True)

In [988]:
#df_users_elite.head()

In [989]:
# def check(value):
#     temp_list = []
#     for x in value:
#         if x == 0:
#             y = 1
#             temp_list.append(y)
#     return temp_list


# df_users_elite['temp_elite'] = df_users_elite['elite'].map(check)    

In [990]:
#df_users_elite['temp_elite'] = df_users_elite['elite'].map(lambda x: eval(x))

In [991]:
#df_users_elite['temp_elite_len'] = df_users_elite['temp_elite'].map(lambda x: len(x))

In [992]:
def elite_check(value):
    value_temp = eval(value)
    value_final = len(value_temp)
    if value_final > 0:
        return 1
    else:
        return 0

df_users_elite['elite_status'] = df_users_elite['elite'].map(elite_check)

In [993]:
###Find the predictors that help define "elite" that aren't based on year
def elite_col_check(df_col):
    service_list = []
    for x in df_col:
        if 'elite' in x or 'year' in x:
            service_list.append(x)
    return service_list

elite_col_check(df_users_elite.columns)

['elite', 'elite_status']

In [994]:
df_users_elite.head()

,yelping_since,compliments_plain,review_count,compliments_cute,compliments_writer,fans,compliments_note,compliments_hot,compliments_cool,compliments_profile,...,elite,name,user_id,votes_cool,compliments_list,votes_funny,compliments_photos,compliments_funny,votes_useful,elite_status
0,2004-10,NaN,6,NaN,NaN,0,NaN,NaN,NaN,NaN,...,[],Brandon,GwnFpsaEaLZfVh844EM0Jg,1,None,0,None,NaN,7,0
1,2004-10,1.0,5,NaN,1.0,1,NaN,NaN,NaN,NaN,...,[],Teel,5KwFISX_GfnFUQ0VvaIE1A,1,None,2,None,NaN,16,0
2,2004-10,1.0,94,NaN,1.0,1,1.0,NaN,NaN,NaN,...,[],Larry,82d-mm3hB2UelHwrptvcMw,8,None,4,None,NaN,17,0
3,2004-10,2.0,66,2.0,2.0,4,1.0,1.0,1.0,NaN,...,[2005],Katherine,SIBCL7HBkrP4llolm4SC2A,13,None,11,None,NaN,34,1
4,2004-10,89.0,442,23.0,24.0,100,83.0,101.0,145.0,9.0,...,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",Michael,4U9kSBLuBDU391x6bxU-YA,732,4.0,908,24.0,120.0,1483,1


In [995]:
###Will take out any non-numeric values now for the predictions
elite_list = ['yelping_since', 'elite', 'name', 'user_id', 'elite_status']
formula = 'elite_status ~ ' + ' + '.join([x for x in df_users_elite.columns if x not in elite_list]) + ' - 1'
#print formula

y, X = patsy.dmatrices(formula, data=df_users_elite, return_type='dataframe')

y = np.ravel(y)
ss = StandardScaler()
Xn = ss.fit_transform(X)

print np.mean(y)

0.938786279683


In [996]:
# from sklearn.neighbors import KNeighborsClassifier
# params = {
#     'n_neighbors':range(1,101),
#     'weights':['uniform','distance']
# }

# knn = KNeighborsClassifier()

# knn_gs = GridSearchCV(knn, params, cv=5, verbose=1)
# knn_gs.fit(Xn, y)

In [997]:
# print knn_gs.best_params_
# best_knn = knn_gs.best_estimator_

In [998]:
# best_knn.fit(Xn, y)
# scores = cross_val_score(best_knn, Xn, y, cv=5)
# print scores
# print np.mean(scores)

In [999]:
# ###Logistic Fit
# from sklearn.svm import SVC

# svc_params = {
#     'C':np.logspace(1e-4, 1e1, 20),
#     'kernel':['linear','rbf']
# }

# svc_gs = GridSearchCV(SVC(), svc_params, cv=5, verbose=1)
# svc_gs.fit(Xn, y)

In [1000]:
# best_svc = svc_gs.best_estimator_
# print svc_gs.best_params_
# print svc_gs.best_score_

In [1001]:
# from sklearn.linear_model import LogisticRegression

# lr_params = {
#     'penalty':['l1','l2']
#     'C':np.linspace(1e-4,1e2, 40)
# }

# lr_gs = GridSearchCV(LogisticRegression(), lr_params, cv=5, verbose=1)
# lr_gs.fit(Xn, y)

In [1002]:
# best_lr = lr_gs.best_estimator_
# print lr_gs.best_params_
# print lr_gs.best_score_

In [1003]:
# coefs = pd.DataFrame({'coef':best_knn.coef_, 'abs_coef':np.abs(best_knn.coef_),
#                       'variable':X.columns})
# coefs.sort_values('abs_coef', ascending=False, inplace=True)
# coefs.head(20)

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Find something interesting on your own

---

You want to impress your superiors at Yelp by doing some investigation into the data on your own. You want to do classification, but you're not sure on what.

1. Create a hypothesis or hypotheses about the data based on whatever you are interested in, as long as it is predicting a category of some kind (classification).
2. Explore the data visually (ideally related to this hypothesis).
3. Build one or more classification models to predict your target variable. **Your modeling should include gridsearching to find optimal model parameters.**
4. Evaluate the performance of your model. Explain why your model may have chosen those specific parameters during the gridsearch process.
5. Write up what the model tells you. Does it validate or invalidate your hypothesis? Write this up as if for a non-technical audience.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. ROC and Precision-recall

---

Some categories have fewer overall businesses than others. Choose two categories of businesses to predict, one that makes your proportion of target classes as even as possible, and another that has very few businesses and thus makes the target varible imbalanced.

1. Create two classification models predicting these categories. Optimize the models and choose variables as you see fit.
- Make confusion matrices for your models. Describe the confusion matrices and explain what they tell you about your models' performance.
- Make ROC curves for both models. What do the ROC curves describe and what do they tell you about your model?
- Make Precision-Recall curves for the models. What do they describe? How do they compare to the ROC curves?
- Explain when Precision-Recall may be preferable to ROC. Is that the case in either of your models?